# 4. Les pixels

In [ ]:
# settings for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# turn off matplotlib figure axes, else we need to call axes('off') everytime
import matplotlib as mpl
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
plt.set_cmap('gray') 
mpl.rc('axes.spines',top=False,bottom=False,left=False,right=False)
mpl.rc(('xtick','ytick'),color=(1,1,1,0))
mpl.rc('font', size=20)

# import necessarry libs
# plugin to read hdr file (this may require internet to download files.)
import imageio; imageio.plugins.freeimage.download()
from imageio import imread

from PIL import Image # another option for image processing
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Image en "niveaux de gris"

In [ ]:
# imageio can read and write image data for a large variety of formats
# most of our demos are using this library
from imageio import imread
img_gray = imread('composite-gray.jpg')
print(type(img_gray), img_gray.shape)

### other options to load images

In [ ]:
from PIL import Image
img_pil = Image.open("composite-gray.jpg")
print(type(img_pil), 'width:', img_pil.height, 'height:', img_pil.width)

### convert an array to the Image structure
```python
img_array = imageio.imread('composite-gray.jpg')
img_pil = Image.fromarray(img_array)
```
### vice versa
```python
img_pil = Image.open("composite-gray.jpg")
img_array = np.asarray(img_pil)
```

In [ ]:
img_arr = np.asarray(img_pil)
print('type:', type(img_arr), 'shape:', img_arr.shape)

# Quel est le type de données dans ce tableau?
print('data type:', img_arr.dtype)
print(img_arr)

In [ ]:
# resize an image
img_gray = imread('composite-gray.jpg')
print(img_gray.shape)

# scikit-image is a collection of algorithms for image processing. 
from skimage.transform import resize
img_gray = resize(img_gray, [400, 600])
print(img_gray.shape)


### another option: use the resize function in PIL
```python
img_pil = Image.fromarray(img_gray).resize([600, 400]) 
```

## show image in python

In [ ]:
# use matplotlib to show image
from matplotlib.pyplot import imshow
%matplotlib notebook
imshow(img_gray);plt.axis('off');

### aonther option: use PIL's show() function, which will open a new window
```python
img_pil.show()
```

# Lecture de ce que la camera capture vraiment

In [ ]:
imgRAW = imread('mandi.tif')

# let's show the image in a larger figure frame
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 6));plt.axis('off')
plt.imshow(imgRAW);

# Interpolation de Bayer

In [ ]:
from colour_demosaicing import demosaicing_CFA_Bayer_bilinear as demosaic
imgBayer = Image.fromarray(demosaic(imgRAW, 'BGGR'))

fig, ax = plt.subplots(1, 2, figsize=(12,5), sharex=True, sharey=True, num='Interactive plot')
ax[0].imshow(imgRAW); ax[0].axis('off'); ax[0].set_title('image RAW');
ax[1].imshow(imgBayer); ax[1].axis('off'); ax[1].set_title('image après interpolation de Bayer')
plt.tight_layout() # this can slightly change the image pattern
plt.show()

# Canaux

In [ ]:
img = imread('img-couleur.jpg');
print('Shape: ', img.shape)

In [ ]:
fig = plt.figure(figsize=(10, 5))
plt.imshow(img); plt.axis('off');

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5), sharex=True, sharey=True, num='RGB Canaux')
plt.set_cmap('gray')  # jet?
ax[0].imshow(img[:,:,0]); ax[0].axis('off'); ax[0].set_title('R')
ax[1].imshow(img[:,:,1]); ax[1].axis('off'); ax[1].set_title('G')
ax[2].imshow(img[:,:,2]); ax[2].axis('off'); ax[2].set_title('B')
plt.tight_layout()
plt.show()

# Comment visualiser "seulement" la teinte?

In [ ]:
img = imread('img-couleur.jpg')
from skimage.color import rgb2hsv, hsv2rgb
imgHsv = rgb2hsv(img)
imgHsv[:,:,1] = 1
imgHsv[:,:,2] = 1

imgModifiee = hsv2rgb(imgHsv);
fig, ax = plt.subplots(1, 2, figsize=(10,5), sharex=True, sharey=True, num='HSV 1')
ax[0].imshow(img); ax[0].axis('off'); ax[0].set_title('Image originale')
ax[1].imshow(imgModifiee); ax[1].axis('off'); ax[1].set_title('Modifiee')
plt.tight_layout()

In [ ]:
imgHsv = rgb2hsv(img)
imgHsv[:,:,1] = imgHsv[:,:,1] * 1.5  # 50% de plus de saturation!
imgHsv[:,:,1][imgHsv[:,:,1]>1] = 1  # maximum saturation should be less than 1

imgModifiee = hsv2rgb(imgHsv);
fig, ax = plt.subplots(1, 2, figsize=(10,5), sharex=True, sharey=True, num='HSV 2')
ax[0].imshow(img); ax[0].axis('off'); ax[0].set_title('Image originale')
ax[1].imshow(imgModifiee); ax[1].axis('off'); ax[1].set_title('Saturation + 50%')
plt.tight_layout()

# Manipuler la saturation interactivement!

In [ ]:
imgRAW = imread('mandi.tif')
from colour_demosaicing import demosaicing_CFA_Bayer_bilinear as demosaic
from skimage.transform import resize
img = demosaic(imgRAW, 'BGGR')
img = resize(img, (400,600))

# show image
fig, ax = plt.subplots(1, 2, figsize=(15,5), sharex=True, sharey=True, num='Saturation')
ax[0].imshow(img); ax[0].axis('off'); ax[0].set_title('Image originale')
ax[1].imshow(img); ax[1].axis('off'); ax[1].set_title('Saturation x 1')
plt.tight_layout()

'''interactive controller'''
from matplotlib.widgets import Slider
ax_slider = plt.axes([0.25, 0, 0.65, 0.05], facecolor='lightblue')
slider = Slider(ax_slider, 'Saturation ratio', 0, 2, valinit=1, valstep=0.05)
imgHsv = rgb2hsv(img) # modify this Hsv iamge
def update_plot(val):
    ax[1].set_title('Saturation x %.2f' % (val))
    _imgHsv = imgHsv.copy()
    _imgHsv[...,1] = np.minimum(1, imgHsv[...,1] * val)
    img_update = hsv2rgb(_imgHsv)
    ax[1].imshow(img_update)
    fig.canvas.draw_idle()
slider.on_changed(update_plot);

# Espace de couleur lab

In [ ]:
img = imread('img-couleur.jpg')

from skimage.color import rgb2lab, lab2rgb
imglab = rgb2lab(img)

# visualisons la luminance et chrominance
imglab = rgb2lab(img)
imgLuminance = imglab[:,:,0]/100
imglab[:,:,0] = 65
imgChrominance = lab2rgb(imglab)

# show image
fig, ax = plt.subplots(1, 2, figsize=(15,5), sharex=True, sharey=True, num='Luminance & Chrominance')
ax[0].imshow(imgLuminance); ax[0].axis('off'); ax[0].set_title('Luminance')
ax[1].imshow(imgChrominance); ax[1].axis('off'); ax[1].set_title('Chrominance')
plt.tight_layout()

# Plage dynamique

In [ ]:
img = imread('memorial.hdr')  # .hdr file contains float values
print('dtype:', img.dtype)
print('range in [%.2f, %.2f]' % (img.min(), img.max()))

# show image
fig, ax = plt.subplots(1, 2, figsize=(12,10), sharex=True, sharey=True, num='HDR')
ax[0].imshow(img); ax[0].axis('off'); ax[0].set_title('Image originale')  # to disable the plot warning do: imshow(np.minimum(1, img * 1))
ax[1].imshow(img * 6); ax[1].axis('off'); ax[1].set_title('Image x 6')
plt.tight_layout()

In [ ]:
img6 = img * 6

%matplotlib notebook
mpl.rc('font', size=20)
fig, ax = plt.subplots(1, 2, figsize=(12,10), sharex=True, sharey=True, num='Interactive plot')
ax[0].imshow(img6); ax[0].axis('off'); ax[0].set_title('Image originale')
ax[1].imshow(img6/(1.0+img6)); ax[1].axis('off'); ax[1].set_title('Reinhard')
plt.tight_layout()

# Égalisation d'histogramme

In [ ]:
# Show axes label from here
# show the plot with labels
mpl.rc(('xtick','ytick'), color=(0,0,0,1))
mpl.rc('axes.spines',top=True,bottom=True,left=True,right=True)

In [ ]:
img = imread('image40.jpg')
imgHist, _bin_edges = np.histogram(img.flatten(), bins=256, range=(0,255))
imgHistCum = np.cumsum(imgHist)

fig, ax = plt.subplots(1, 2, figsize=(15,5), num='Plot')

ax[0].plot(imgHist); ax[0].grid(); ax[0].set_title('Histogramme')
ax[1].plot(imgHistCum); ax[1].grid(); ax[1].set_title('Histogramme cumulatif')

plt.tight_layout()

In [ ]:
N = img.shape[0] * img.shape[1]
imgEq = (imgHistCum[img].astype('float32')*255./float(N)).astype('uint8')

# show the plot
fig, ax = plt.subplots(2, 2, figsize=(10,5), num='Image & Histogramme')
ax[0][0].imshow(img); ax[0][0].set_title('Image'); ax[0][0].axis('off')
ax[0][1].hist(img.flatten(), 255)[0]; ax[0][1].grid(); ax[0][1].set_title('Image Histogramme')
ax[1][0].imshow(imgEq); ax[1][0].grid(); ax[1][0].set_title('imgEq'); ax[1][0].axis('off')
ax[1][1].hist(imgEq.flatten(), 255)[0]; ax[1][1].grid(); ax[1][1].set_title('imgEq Histogramme')
plt.set_cmap('gray')
plt.tight_layout()

# On le fait pour une image couleur?

In [ ]:
imgIn = imread('arbres.jpg').astype('float32') / 255.0  # here we convert uint8 to float

In [ ]:
# Enhance contrast using histogram equalization
def histeq(img):
    imgHist = np.histogram(img.flatten(), bins=256, range=(0,1))[0]
    imgHistCum = np.cumsum(imgHist)
    N = img.shape[0] * img.shape[1]
    imgEq = imgHistCum[(img*255).astype(int)]/N
    return imgEq

In [ ]:
# option 1: chaque canal independamment
imgEq1 = np.zeros(imgIn.shape)
for i_c in range(3):
    imgEq1[:,:,i_c] = histeq(imgIn[:,:,i_c])

In [ ]:
# option 2: equilibrer le canal 'L' seulement 
from skimage.color import rgb2lab, lab2rgb

imgLab = rgb2lab(imgIn);
L = imgLab[:,:,0]
# L est entre 0 et 100. mettre entre 0 et 255 pour histeq
L = L/100.0

L = histeq(L)
L = L * 100.0
imgLab[:,:,0] = L
imgEq2 = lab2rgb(imgLab);

In [ ]:
# show the images
alpha = .5
fig, ax = plt.subplots(1, 3, figsize=(15,5), sharex=True, sharey=True)
ax[0].imshow(imgIn); ax[0].axis('off'); ax[0].set_title('Image originale')
ax[1].imshow(imgEq1*alpha + (1-alpha)*imgIn); ax[1].axis('off'); ax[1].set_title('Canaux indépendants')
ax[2].imshow(imgEq2*alpha + (1-alpha)*imgIn); ax[2].axis('off'); ax[2].set_title('L seulement');